# Beagle

## Run Webserver to handle queries

In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import threading
from src.embedder import Embedder
from src.search_engine import SearchEngine
from src.database_manager import DatabaseManager
from src.content_manager import ContentManager


# Create a Flask application
app = Flask(__name__)

# Enable CORS for all routes
CORS(app)

# Don't be fooled by the name. The embeddings are float16 tensors.
embedder = Embedder("ViT-B/32")
database_manager = DatabaseManager()

engine = SearchEngine(database_manager)

content_manager = ContentManager(database_manager, embedder)

content_manager.process_input_file("scope.txt")

# Define the search route
@app.route('/search', methods=['GET'])
def search():
    query = request.args.get('query', '')
    query_embedding = embedder.embedText(query)
    
    return jsonify(engine.search_for(query_embedding))

def run_app():
    app.run(port=5000)

# Run the Flask app in a separate thread
flask_thread = threading.Thread(target=run_app)
flask_thread.start()
